In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-201896
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-201896


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cluster1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)
aml_compute.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
##ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling( {
        '--C' : choice(0.001,0.01,0.1,1.0,10.0,50.0,100,1000),
        '--max_iter': choice(50,100)
    }
)

# Specify a Policy
##policy = ### YOUR CODE HERE ###
etpolicy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
##src = ### YOUR CODE HERE ###
src=ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=aml_compute,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
##hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=etpolicy,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e1c90235-2a21-4ce9-8d25-3b44e80842c6
Web View: https://ml.azure.com/runs/HD_e1c90235-2a21-4ce9-8d25-3b44e80842c6?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-201896/workspaces/quick-starts-ws-201896&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-07-23T13:25:21.101386][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-07-23T13:25:21.9541696Z][SCHEDULER][INFO]Scheduling job, id='HD_e1c90235-2a21-4ce9-8d25-3b44e80842c6_0' 
[2022-07-23T13:25:22.1386729Z][SCHEDULER][INFO]Scheduling job, id='HD_e1c90235-2a21-4ce9-8d25-3b44e80842c6_1' 
[2022-07-23T13:25:22.2228468Z][SCHEDULER][INFO]Scheduling job, id='HD_e1c90235-2a21-4ce9-8d25-3b44e80842c6_2' 
[2022-07-23T13:25:22.3533140Z][SCHEDULER][INFO]Scheduling job, id='HD_e1c90235-2a21-4ce9-8d25-3b44e80842c6_3' 
[2022-07-23T13:25:22.294979][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_e1c90235-2a21-4ce9-8d25-3b44e80842c6',
 'target': 'cluster1',
 'status': 'Completed',
 'startTimeUtc': '2022-07-23T13:25:20.328309Z',
 'endTimeUtc': '2022-07-23T13:44:40.932855Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'c86e33e8-eb36-4ab5-bc9b-eaa27ddb47f2',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1083-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.42.0',
  'space_size': '16',
  'score': '0.9127215343529983',
  'best_child_run_id': 'HD_e1c90235-2a21-4ce9-8d25-3b44e80842c6_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_e1c90235-2a21-4ce9-8d25-3b44e80842c6_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType

In [32]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics=best_run.get_metrics()
print(best_run.get_file_names())
print(best_run_metrics)
#best_run.register_model(model_name='hmodel',model_path='outputs/model.joblib')
##joblib.dump(best_run_model, './hmodel.joblib')

['azureml-logs/20_image_build_log.txt', 'logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2022-07-23-13', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'user_logs/std_log.txt']
{'Regularization Strength:': 0.001, 'Max iterations:': 100, 'Accuracy': 0.9127215343529983}


In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')


In [11]:
from train import clean_data

# Use the clean_data function to clean your data.
##x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(ds)

In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [13]:
# Submit your automl run

### YOUR CODE HERE ###
run = exp.submit(config=automl_config, show_output=False)
RunDetails(run).show()
run.wait_for_completion()

2022-07-23:14:30:28,411 WARNING  [local_experiment_driver.py:145] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_4584287c-3362-44b6-a4d6-7e32db1457cb,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2022-07-23:15:06:13,359 INFO     [explanation_client.py:334] Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_4584287c-3362-44b6-a4d6-7e32db1457cb',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-07-23T14:30:33.761898Z',
 'endTimeUtc': '2022-07-23T15:04:10.772721Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-201896","workspace_name":"quick-starts-ws-201896","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,

In [17]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run_auto=run.get_best_child()
best_run_model_name=best_run_auto.properties["model_name"]
run.register_model(model_name=best_run_model_name,description='automl model',tags=None)
   
#automl_model = run.get_output()
#run.get_metrics()
#run.register_model(description='automl model')

Model(workspace=Workspace.create(name='quick-starts-ws-201896', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-201896'), name=AutoML4584287c331, id=AutoML4584287c331:1, version=1, tags={}, properties={})

In [33]:
#Delete compute
aml_compute.delete()